
# DDPG 演算法實作（Pendulum-v1 環境）- Google Colab 版

本 Notebook 整合了完整的 DDPG 類別、訓練流程、測試程式，並支援 Gym 新版 API。


In [ ]:

!pip install tensorlayer==2.2.3 tensorflow-probability==0.6.0 gym pygame


In [ ]:

import os, time
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt
import tensorlayer as tl

ENV_NAME = 'Pendulum-v1'
RANDOMSEED = 1

LR_A = 0.001
LR_C = 0.002
GAMMA = 0.9
TAU = 0.01
MEMORY_CAPACITY = 10000
BATCH_SIZE = 32

MAX_EPISODES = 200
MAX_EP_STEPS = 200
TEST_PER_EPISODES = 10
VAR = 3


In [ ]:

env = gym.make(ENV_NAME)
env.reset(seed=RANDOMSEED)
np.random.seed(RANDOMSEED)
tf.random.set_seed(RANDOMSEED)

s_dim = env.observation_space.shape[0]
a_dim = env.action_space.shape[0]
a_bound = env.action_space.high

print('s_dim:', s_dim)
print('a_dim:', a_dim)


In [ ]:

class DDPG(object):
    def __init__(self, a_dim, s_dim, a_bound):
        self.memory = np.zeros((MEMORY_CAPACITY, s_dim * 2 + a_dim + 1), dtype=np.float32)
        self.pointer = 0
        self.a_dim, self.s_dim, self.a_bound = a_dim, s_dim, a_bound

        W_init = tf.random_normal_initializer(mean=0, stddev=0.3)
        b_init = tf.constant_initializer(0.1)

        def get_actor(input_state_shape, name=''):
            inputs = tl.layers.Input(input_state_shape, name='A_input')
            x = tl.layers.Dense(n_units=30, act=tf.nn.relu, W_init=W_init, b_init=b_init, name='A_l1')(inputs)
            x = tl.layers.Dense(n_units=a_dim, act=tf.nn.tanh, W_init=W_init, b_init=b_init, name='A_a')(x)
            x = tl.layers.Lambda(lambda x: a_bound * x)(x)
            return tl.models.Model(inputs=inputs, outputs=x, name='Actor' + name)

        def get_critic(input_state_shape, input_action_shape, name=''):
            s = tl.layers.Input(input_state_shape, name='C_s_input')
            a = tl.layers.Input(input_action_shape, name='C_a_input')
            x = tl.layers.Concat(1)([s, a])
            x = tl.layers.Dense(n_units=60, act=tf.nn.relu, W_init=W_init, b_init=b_init, name='C_l1')(x)
            x = tl.layers.Dense(n_units=1, W_init=W_init, b_init=b_init, name='C_out')(x)
            return tl.models.Model(inputs=[s, a], outputs=x, name='Critic' + name)

        self.actor = get_actor([None, s_dim])
        self.critic = get_critic([None, s_dim], [None, a_dim])
        self.actor.train()
        self.critic.train()

        def copy_para(from_model, to_model):
            for i, j in zip(from_model.trainable_weights, to_model.trainable_weights):
                j.assign(i)

        self.actor_target = get_actor([None, s_dim], name='_target')
        self.critic_target = get_critic([None, s_dim], [None, a_dim], name='_target')
        copy_para(self.actor, self.actor_target)
        copy_para(self.critic, self.critic_target)
        self.actor_target.eval()
        self.critic_target.eval()

        self.ema = tf.train.ExponentialMovingAverage(decay=1 - TAU)
        self.actor_opt = tf.optimizers.Adam(LR_A)
        self.critic_opt = tf.optimizers.Adam(LR_C)

    def ema_update(self):
        paras = self.actor.trainable_weights + self.critic.trainable_weights
        self.ema.apply(paras)
        for i, j in zip(self.actor_target.trainable_weights + self.critic_target.trainable_weights, paras):
            i.assign(self.ema.average(j))

    def choose_action(self, s):
        return self.actor(np.array([s], dtype=np.float32))[0]

    def learn(self):
        indices = np.random.choice(MEMORY_CAPACITY, size=BATCH_SIZE)
        bt = self.memory[indices, :]
        bs = bt[:, :self.s_dim]
        ba = bt[:, self.s_dim:self.s_dim + self.a_dim]
        br = bt[:, -self.s_dim - 1:-self.s_dim]
        bs_ = bt[:, -self.s_dim:]

        with tf.GradientTape() as tape:
            a_ = self.actor_target(bs_)
            q_ = self.critic_target([bs_, a_])
            y = br + GAMMA * q_
            q = self.critic([bs, ba])
            td_error = tf.losses.mean_squared_error(y, q)
        c_grads = tape.gradient(td_error, self.critic.trainable_weights)
        self.critic_opt.apply_gradients(zip(c_grads, self.critic.trainable_weights))

        with tf.GradientTape() as tape:
            a = self.actor(bs)
            q = self.critic([bs, a])
            a_loss = -tf.reduce_mean(q)
        a_grads = tape.gradient(a_loss, self.actor.trainable_weights)
        self.actor_opt.apply_gradients(zip(a_grads, self.actor.trainable_weights))

        self.ema_update()

    def store_transition(self, s, a, r, s_):
        s = s.astype(np.float32)
        s_ = s_.astype(np.float32)
        transition = np.hstack((s, a, [r], s_))
        index = self.pointer % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.pointer += 1

    def save_ckpt(self):
        if not os.path.exists('model'):
            os.makedirs('model')
        tl.files.save_weights_to_hdf5('model/ddpg_actor.hdf5', self.actor)
        tl.files.save_weights_to_hdf5('model/ddpg_actor_target.hdf5', self.actor_target)
        tl.files.save_weights_to_hdf5('model/ddpg_critic.hdf5', self.critic)
        tl.files.save_weights_to_hdf5('model/ddpg_critic_target.hdf5', self.critic_target)

    def load_ckpt(self):
        tl.files.load_hdf5_to_weights_in_order('model/ddpg_actor.hdf5', self.actor)
        tl.files.load_hdf5_to_weights_in_order('model/ddpg_actor_target.hdf5', self.actor_target)
        tl.files.load_hdf5_to_weights_in_order('model/ddpg_critic.hdf5', self.critic)
        tl.files.load_hdf5_to_weights_in_order('model/ddpg_critic_target.hdf5', self.critic_target)


In [ ]:
ddpg = DDPG(a_dim, s_dim, a_bound)

In [ ]:

reward_buffer = []
t0 = time.time()
for i in range(MAX_EPISODES):
    s, _ = env.reset()
    ep_reward = 0
    for j in range(MAX_EP_STEPS):
        a = ddpg.choose_action(s)
        a = np.clip(np.random.normal(a, VAR), -2, 2)
        s_, r, terminated, truncated, _ = env.step(a)
        done = terminated or truncated

        ddpg.store_transition(s, a, r / 10, s_)
        if ddpg.pointer > MEMORY_CAPACITY:
            ddpg.learn()

        s = s_
        ep_reward += r
        if done:
            break

    if i and not i % TEST_PER_EPISODES:
        print(f"Episode: {i}, Reward: {ep_reward:.2f}")
        reward_buffer.append(ep_reward)

ddpg.save_ckpt()
print("Training done in %.2f seconds." % (time.time() - t0))


In [ ]:

ddpg.load_ckpt()

for episode in range(3):
    s, _ = env.reset()
    ep_reward = 0
    for step in range(MAX_EP_STEPS):
        a = ddpg.choose_action(s)
        s, r, terminated, truncated, _ = env.step(a)
        ep_reward += r
        if terminated or truncated:
            break
    print(f"[測試] Episode {episode + 1} reward: {ep_reward:.2f}")
